In [1]:
import os
import json
import traceback
import pandas as pd

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()     #loads environment variables from .env file

True

In [3]:
KEY = os.getenv("OPEN_API_KEY")
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5)

/Users/tryambakadiga/Desktop/mcqgen/name/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


sk-proj-k5TEr3m3x1Urpf8PAa4YT3BlbkFJsLPzDOA933orb5x14HBw


In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [5]:
RESPONSE_JSON = {
    "1":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    },
    "2":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    },
    "3":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    },
}

In [6]:
TEMPLATE = """
Text : {text}
You are an expert MCQ maker , given the above text, it is your job to create a quiz of {number} of multiple choice
questions {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming of the text.
Make sure to format your response like RESPONSE_JSON below and use it as a guide
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE 
)

In [8]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key= "quiz", verbose = True)

/Users/tryambakadiga/Desktop/mcqgen/name/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
TEMPLATE_2 = """
You are an expert english grammarian and writer , Given a multiple choice quiz on {subject} students 
you need to evaluate the complexity of the questions and give a complete analysis of the quiz.
Use less than 50 words for the analysis. If the quiz is not at par with the cognitive abilities of the students then change 
the tone such that it perfectly fits the student's abilities
QUIZ MCQ:
{quiz}

"""

In [10]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE_2
)

In [11]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [12]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain, review_chain],
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    output_variables = ["quiz", "review"],
    verbose = True
)

In [13]:
# read data from file 
file_path = r"/Users/tryambakadiga/Desktop/mcqgen/data.txt"
with open (file_path, 'r') as file:
    TEXT = file.read()

In [14]:
print(TEXT)

Real Madrid Club de Fútbol (Spanish pronunciation: [reˈal maˈðɾið ˈkluβ ðe ˈfuðβol] ⓘ), commonly referred to as Real Madrid, is a Spanish professional football club based in Madrid. The club competes in La Liga, the top tier of Spanish football.

Founded in 1902 as Madrid Football Club, the club has traditionally worn a white home kit since its inception. The honorific title real is Spanish for "royal" and was bestowed to the club by King Alfonso XIII in 1920 together with the royal crown in the emblem. Real Madrid have played their home matches in the 85,000-capacity[7] Santiago Bernabéu in central Madrid since 1947. Unlike most European sporting entities, Real Madrid's members (socios) have owned and operated the club throughout its history. The official Madrid anthem is the "Hala Madrid y nada más", written by RedOne and Manuel Jabois.[8] The club is one of the most widely supported in the world, and is the most followed football club on social media according to the CIES Football O

In [15]:
NUMBER = 5
SUBJECT = "real madrid"
TONE = "simple"

In [16]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain({
        "text" : TEXT,
        "number" : NUMBER,
        "subject" : SUBJECT,
        "tone" : TONE,
        "response_json" : json.dumps(RESPONSE_JSON)
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Real Madrid Club de Fútbol (Spanish pronunciation: [reˈal maˈðɾið ˈkluβ ðe ˈfuðβol] ⓘ), commonly referred to as Real Madrid, is a Spanish professional football club based in Madrid. The club competes in La Liga, the top tier of Spanish football.

Founded in 1902 as Madrid Football Club, the club has traditionally worn a white home kit since its inception. The honorific title real is Spanish for "royal" and was bestowed to the club by King Alfonso XIII in 1920 together with the royal crown in the emblem. Real Madrid have played their home matches in the 85,000-capacity[7] Santiago Bernabéu in central Madrid since 1947. Unlike most European sporting entities, Real Madrid's members (socios) have owned and operated the club throughout its history. The official Madrid anthem is the "Hala Madrid y nada más", written by RedOne and Manuel Jabois.[8] The club is one of the most widely s

/Users/tryambakadiga/Desktop/mcqgen/name/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer , Given a multiple choice quiz on real madrid students 
you need to evaluate the complexity of the questions and give a complete analysis of the quiz.
Use less than 50 words for the analysis. If the quiz is not at par with the cognitive abilities of the students then change 
the tone such that it perfectly fits the student's abilities
QUIZ MCQ:
{
    "1": {
        "mcq": "When was Real Madrid Club de Fútbol founded?",
        "options": {
            "a": "1900",
            "b": "1902",
            "c": "1920",
            "d": "1947"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "What is the meaning of the honorific title 'real' in Real Madrid?",
        "options": {
            "a": "Royal",
            "b": "Great",
            "c": "Strong",
            "d": "Fast"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "W

In [17]:
print(cb)

Tokens Used: 1592
	Prompt Tokens: 1181
	Completion Tokens: 411
Successful Requests: 2
Total Cost (USD): $0.0025935000000000003


In [18]:
response

{'text': 'Real Madrid Club de Fútbol (Spanish pronunciation: [reˈal maˈðɾið ˈkluβ ðe ˈfuðβol] ⓘ), commonly referred to as Real Madrid, is a Spanish professional football club based in Madrid. The club competes in La Liga, the top tier of Spanish football.\n\nFounded in 1902 as Madrid Football Club, the club has traditionally worn a white home kit since its inception. The honorific title real is Spanish for "royal" and was bestowed to the club by King Alfonso XIII in 1920 together with the royal crown in the emblem. Real Madrid have played their home matches in the 85,000-capacity[7] Santiago Bernabéu in central Madrid since 1947. Unlike most European sporting entities, Real Madrid\'s members (socios) have owned and operated the club throughout its history. The official Madrid anthem is the "Hala Madrid y nada más", written by RedOne and Manuel Jabois.[8] The club is one of the most widely supported in the world, and is the most followed football club on social media according to the CI

In [19]:
print(response.get('review'))

The quiz on Real Madrid is suitable for students with basic knowledge of the football club. The questions are straightforward and test basic facts about the team's history, stadium, and achievements. It is a good way to engage students in learning about sports.


In [20]:
print(response.get('quiz'))

{
    "1": {
        "mcq": "When was Real Madrid Club de Fútbol founded?",
        "options": {
            "a": "1900",
            "b": "1902",
            "c": "1920",
            "d": "1947"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "What is the meaning of the honorific title 'real' in Real Madrid?",
        "options": {
            "a": "Royal",
            "b": "Great",
            "c": "Strong",
            "d": "Fast"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "Where does Real Madrid play their home matches?",
        "options": {
            "a": "Camp Nou",
            "b": "Santiago Bernabéu",
            "c": "Wembley Stadium",
            "d": "Old Trafford"
        },
        "correct": "b"
    },
    "4": {
        "mcq": "How many La Liga titles has Real Madrid won?",
        "options": {
            "a": "20",
            "b": "30",
            "c": "36",
            "d": "40"
        },
        "correct": "c"
    },
  